Using pdf2htmlEX to convert html(see link below) than using it to parse.

using this image with command :
`docker pull pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64`

in detail : [pdf2tmlEX](https://github.com/pdf2htmlEX/pdf2htmlEX)

In [57]:
from unstructured_inference.inference.layout import DocumentLayout
from unstructured_inference.models.base import get_model
from unstructured_inference.inference.layout import process_file_with_model
fname = "example/sample_file.pdf"

model = get_model("yolox")
layout = DocumentLayout.from_file(fname, detection_model=model) # assume you using html parsed file.

In [58]:
annotation_data_map = {
    "final": None,
    "extracted": {"layout": {"color": "green", "width": 2}},
    "inferred": {"inferred_layout": {"color": "blue", "width": 2}},
    "ocr": {"ocr_layout": {"color": "yellow", "width": 2}},
}

In [59]:
for page in layout.pages:
    elems = page.elements

    print(elems)

[LayoutElement(bbox=Rectangle(x1=107.00845336914062, y1=60.57137680053711, x2=302.83349609375, y2=101.11221313476562), text=None, source=<Source.YOLOX: 'yolox'>, type='Page-header', prob=0.34800100326538086, image_path=None, parent=None), LayoutElement(bbox=Rectangle(x1=117.78240203857422, y1=420.69964599609375, x2=1563.4102783203125, y2=477.9920654296875), text=None, source=<Source.YOLOX: 'yolox'>, type='Section-header', prob=0.5629148483276367, image_path=None, parent=None), LayoutElement(bbox=Rectangle(x1=258.0867614746094, y1=511.06005859375, x2=1387.489501953125, y2=583.8973388671875), text=None, source=<Source.YOLOX: 'yolox'>, type='List-item', prob=0.4687464237213135, image_path=None, parent=None), LayoutElement(bbox=Rectangle(x1=234.56793212890625, y1=634.2048950195312, x2=1411.3292236328125, y2=711.9740600585938), text=None, source=<Source.YOLOX: 'yolox'>, type='List-item', prob=0.49582821130752563, image_path=None, parent=None), LayoutElement(bbox=Rectangle(x1=221.23504638671

In [61]:
doc = process_file_with_model(
    fname,
    model_name=None,
)

In [65]:
from unstructured_inference.visualize import show_plot
from unstructured_inference.inference.elements import ImageTextRegion
from unstructured_inference.inference.layout import process_file_with_model
from unstructured_inference.utils import write_image
import os

def run(f_path, scope):
    annotation_data_map = {
        "final": None,
        "extracted": {"layout": {"color": "green", "width": 2}},
        "inferred": {"inferred_layout": {"color": "blue", "width": 2}},
    }

    f_basename = os.path.splitext(os.path.basename(f_path))[0]
    output_dir_path = os.path.join(output_basedir_path, f_basename)
    os.makedirs(output_dir_path, exist_ok=True)

    doc = process_file_with_model(
        f_path,
        model_name=None,
    )

    for idx, page in enumerate(doc.pages):
        if scope == "image_only":
            embedded_image_elements = [
                el for el in page.layout if isinstance(el, ImageTextRegion)
            ]
            inferred_image_elements = [
                el for el in page.inferred_layout if el.type == "Figure"
            ]
            final_image_elements = [el for el in page.elements if el.type == "Image"]

            page.layout = embedded_image_elements
            page.inferred_layout = inferred_image_elements
            page.elements = final_image_elements

        for action_type, action_value in annotation_data_map.items():
            img = page.annotate(annotation_data=action_value)
            output_f_path = os.path.join(output_dir_path, f"{f_basename}_{idx+1}_{action_type}.jpg")
            write_image(img, output_f_path)

        print(f"page_num: {idx+1} - n_total_elements: {len(page.elements)} - n_extracted_elements: "
              f"{len(page.layout)} - n_inferred_elements: {len(page.inferred_layout)}")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [ ]:
import sys
import os
import pathlib

CUR_DIR = pathlib.Path(__file__).parent.resolve()

if len(sys.argv) < 3:
        print(
            "Please provide the path to the file name as the first argument and the scope as the "
            "second argument.",
        )
        sys.exit(1)

if sys.argv[2] not in ["all", "image_only"]:
    print("Invalid scope")
    sys.exit(1)

output_basedir_path = os.path.join(CUR_DIR, "output")
os.makedirs(output_basedir_path, exist_ok=True)

run(f_path=sys.argv[1], scope=sys.argv[2])